In [2]:
import os
from pathlib import Path
import random

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from PIL import Image
from tqdm.auto import tqdm

import torchvision.transforms as T
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights


c:\Users\Nolan\Documents\My dokuments\GoIT\Deep_learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
IS_KAGGLE = False

if IS_KAGGLE:
    COMP_PATH = Path("../input/deep-learning-for-computer-vision-and-nlp-2026-01")
    FOLDS_PATH = Path("../input/petfinder-train-folds/train_folds.csv")  # зміниться під твою назву на Kaggle
else:
    COMP_PATH = Path("../data/raw")
    FOLDS_PATH = Path("../data/artifacts/train_folds.csv")

TRAIN_CSV = COMP_PATH / "train.csv"
TEST_CSV  = COMP_PATH / "test.csv"
IMG_DIR   = COMP_PATH / "images"

print("train exists:", TRAIN_CSV.exists())
print("test exists :", TEST_CSV.exists())
print("images dir  :", IMG_DIR.exists())
print("folds exists:", FOLDS_PATH.exists())


train exists: True
test exists : True
images dir  : True
folds exists: True


In [9]:
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)
folds_df = pd.read_csv(FOLDS_PATH)

print(train_df.shape, test_df.shape, folds_df.shape)
train_df.head()


(6431, 3) (1891, 2) (6431, 3)


,PetID,Description,AdoptionSpeed
0,d3b4f29f8,Mayleen and Flo are two lovely adorable sister...,2
1,e9dc82251,A total of 5 beautiful Tabbys available for ad...,2
2,8111f6d4a,Two-and-a-half month old girl. Very manja and ...,2
3,693a90fda,Neil is a healthy and active ~2-month-old fema...,2
4,9d08c85ef,Gray kitten available for adoption in sungai p...,2


In [11]:
train_df = train_df.merge(folds_df, on=["PetID", "AdoptionSpeed"], how="inner")
print("after merge:", train_df.shape)
train_df[["PetID","AdoptionSpeed","fold"]].head()


after merge: (6431, 4)


,PetID,AdoptionSpeed,fold
0,d3b4f29f8,2,3
1,e9dc82251,2,3
2,8111f6d4a,2,2
3,693a90fda,2,4
4,9d08c85ef,2,3


In [12]:
all_imgs = list(IMG_DIR.glob("*"))
print("total image files:", len(all_imgs))
print("sample:", [p.name for p in all_imgs[:10]])

total image files: 1
sample: ['images']
